In [1]:
import requests
from datetime import datetime
import os
import time
from bs4 import BeautifulSoup
import re
import csv
import schedule

In [2]:
def download_image(x_times):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0'
    }
    image_url = 'https://www.805webcams.com/cameras/cachumalake/camera.jpg'
    base_dir = 'C:\\Users\\jackm\\Desktop\\projects\\fog_predictor\\Images'
    
    for i in range(x_times):
        try:
            image_response = requests.get(image_url, headers=headers, stream=True)
            if image_response.status_code == 200:
                current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                filename = f"{current_time}.jpg"
                full_path = os.path.join(base_dir, filename)
                
                with open(full_path, 'wb') as f:
                    for chunk in image_response:
                        f.write(chunk)
                print(f"Image downloaded successfully and saved as {full_path}!")
            else:
                print(f"Failed to download the image. Status code: {image_response.status_code}")
            if x_times > 1:
                time.sleep(300)  # Delays for 5 minutes
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")



In [3]:
#download_image(1)  # Adjust the number as needed

In [4]:
def scrape_weather_info():
    url = 'https://www.localconditions.com/weather-lake-cachuma-california/93464/'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0'
    }

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')
                
            # Find the element by ID
            weather_text_element = soup.find(id='readLCWXtxt')
            if weather_text_element:
                return weather_text_element.get_text(strip=True)
            else:
                print("Weather information element not found.")
        else:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")


In [5]:
def write_to_csv(data, filename='weather_data.csv'):
    fieldnames = ['Observation Time', 'Temperature', 'Humidity', 'Dew Point', 'Trend', 'Fog']
    
    # Check if file exists to determine if we need to write headers
    try:
        with open(filename, 'x', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerow(dict(zip(fieldnames, data)))
    except FileExistsError:
        with open(filename, 'a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writerow(dict(zip(fieldnames, data)))

In [6]:
def parse_weather_report(text):
    # Regular expressions to find relevant data and observation time
    temperature_pattern = r"It is (\d+) degrees fahrenheit"
    humidity_pattern = r"humidity is (\d+\.?\d*) percent"
    dew_point_pattern = r"dew point of (\d+\.?\d*) degrees fahrenheit"
    trend_pattern = r"that is (\w+) since the last report"
    
    # Extract data using regular expressions
    temperature = re.search(temperature_pattern, text)
    humidity = re.search(humidity_pattern, text)
    dew_point = re.search(dew_point_pattern, text)
    trend = re.search(trend_pattern, text)
    
    # Record time and date of observation
    observation_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Create a list to store the data
    data = [observation_time]

    data.append(temperature.group(1) + ' °F')
    data.append(humidity.group(1) + ' %')
    data.append(dew_point.group(1) + ' °F')
    data.append(trend.group(1))

    # Add an empty column for 'Fog' as a binary
    data.append('')  # Empty string for manual update
    
    return data




In [7]:
def WeatherScript(x_times):
    for i in range(x_times):
        parsed_data=parse_weather_report(scrape_weather_info())
        write_to_csv(parsed_data)
        print("Data has been written to CSV.")
        if x_times > 1:
            time.sleep(300)  # Delays for 5 minutes

In [8]:
def getData(x_times): # 24 for 2 hours
    for i in range(x_times):
        WeatherScript(1)
        download_image(1)
        if x_times > 1:
            time.sleep(60)

In [9]:
def run_at_time():
    print(f"Schedule started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    # Schedule getData to start at 4:45 AM with 24 cycles (adjust as needed)
    schedule.every().day.at("04:45").do(getData, x_times=45)
    while True:
        schedule.run_pending()
        time.sleep(60)  # Check every minute for scheduled tasks

In [10]:
run_at_time()

Schedule started at 2024-05-28 00:25:51
Data has been written to CSV.
Image downloaded successfully and saved as C:\Users\jackm\Desktop\projects\fog_predictor\Images\2024-05-28_04-45-55.jpg!
Data has been written to CSV.
Image downloaded successfully and saved as C:\Users\jackm\Desktop\projects\fog_predictor\Images\2024-05-28_04-46-57.jpg!
Data has been written to CSV.
Image downloaded successfully and saved as C:\Users\jackm\Desktop\projects\fog_predictor\Images\2024-05-28_04-47-59.jpg!
Data has been written to CSV.
Image downloaded successfully and saved as C:\Users\jackm\Desktop\projects\fog_predictor\Images\2024-05-28_04-49-01.jpg!
Data has been written to CSV.
Image downloaded successfully and saved as C:\Users\jackm\Desktop\projects\fog_predictor\Images\2024-05-28_04-50-03.jpg!
Data has been written to CSV.
Image downloaded successfully and saved as C:\Users\jackm\Desktop\projects\fog_predictor\Images\2024-05-28_04-51-05.jpg!
Data has been written to CSV.
Image downloaded success